In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import fastai

from fastai.io import *

from fastai.column_data import *
from fastai.text import *


from pathlib import Path

/home/nrhodes/.local/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/home/nrhodes/.local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Learning to generate Dickens-like text

### Setup

In [2]:
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

PATH=Path('../data/dickens/')

TRN_PATH = 'trn'
VAL_PATH = 'val'
TRN = PATH / TRN_PATH
VAL = PATH / VAL_PATH

%ls {PATH}

davidcopperfield.txt  models/  tmp/  trn/  val/


In [3]:
TEXT = data.Field(lower=True, tokenize=list)
bs=2048; bptt=16; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(44, 53, 1, 1494913)

### Our model

In [4]:
class CharSequence(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.dropout = nn.Dropout(.1)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.0)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.dropout(self.e(cs)), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [5]:
m = BasicModel(CharSequence(md.nt, n_fac, n_hidden, 1).cuda())
learner = RNN_Learner(md, m, opt_fn=optim.Adam, crit=F.nll_loss)

#minimum_learning_rate_divisor = 1200
#percent_after_triangle_cycle = 15
#max_momentum=.97
#min_momentum=.85
#learner.fit(1e-2, 1, cycle_len=72, 
#           use_clr_beta=(minimum_learning_rate_divisor, 
#                         percent_after_triangle_cycle, 
#                         max_momentum, 
#                         min_momentum),
#                         wds=1e-5)
learner.fit(2e-2, 7, cycle_mult=2, cycle_len=1, 
           wds=1e-5)


epoch      trn_loss   val_loss   
    0      2.572774   2.276912  
    1      2.115803   1.784056  
    2      1.863651   1.711806  
    3      1.722404   1.592232  
    4      1.609507   1.538926  
    5      1.540361   1.505572  
    6      1.493371   1.499538  
    7      1.488782   1.507937  
    8      1.46603    1.473991  
    9      1.437556   1.456916  
    10     1.412901   1.440341  
    11     1.391879   1.428559  
    12     1.373634   1.414256  
    13     1.361996   1.406121  
    14     1.350272   1.409307  
    15     1.380842   1.452749  
    16     1.383146   1.434688  
    17     1.376513   1.428363  
    18     1.368166   1.425418  
    19     1.358563   1.404775  
    20     1.348451   1.404826  
    21     1.338716   1.393615  
    22     1.328626   1.386471  
    23     1.320264   1.388079  
    24     1.311041   1.379866  
    25     1.303094   1.374781  
    26     1.294717   1.368882  
    27     1.285488   1.362586  
    28     1.278437   1.361251  
    29   

[array([1.31782])]

### Test

In [6]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = learner.model(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [7]:
get_next('for thos')
#TEXT

'e'

In [8]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [9]:
print(get_next_n('fourscore and seven years ago', 400))

fourscore and seven years ago, as a repelled youngman, comes, claraves, peggotty possessed as to the ground falls street and kind of a tea?)‘i will not help that. you must be disorder that you read, after boys to speak rounded, my mother, didn’t be taken to us you, sir,’ said my aunt. ‘they were notexprause a nighty more. my love from an agreeable life, inexpected myself, then when i am often could be happy himselfback afore 
